In [ ]:
# Original code given by Coin Market Cap website

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
# Adjusted code w my API and production URL

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '42a9d404-6f39-4a6d-8533-520dbc771c80',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
# Identifying data type

type(data)

In [ ]:
import pandas as pd

In [ ]:
# Displays full array of columns rather than a summary

pd.set_option('display.max_columns', None)

In [ ]:
# Normalizes the data and makes it simpler to read

pd.json_normalize(data['data'])

In [ ]:
# Creating df and adding a time stamp for the data

df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')

df

In [ ]:
# Creating a function that runs the api

def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '42a9d404-6f39-4a6d-8533-520dbc771c80',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    df2 = pd.json_normalize(data['data'])
    df2['timestamp'] = pd.to_datetime('now')
    df_append = pd.DataFrame(df2)

    df = pd.concat([df, df_append])
    
# Alternative option for CSV conversion rather than being kept in the df
#    df2 = pd.json_normalize(data['data'])
#    df2['timestamp'] = pd.to_datetime('now')
#    df_append = pd.DataFrame(df2)
#    df

#    if not os.path.isfile(r'C:\Users\Joven\Desktop\Portfolio\Automated Crypto API Pull\API.csv')
#        df.to_csv(r'C:\Users\Joven\Desktop\Portfolio\Automated Crypto API Pull\API.csv', header = 'column_names')
#    else:
#        df.to_csv(r'C:\Users\Joven\Desktop\Portfolio\Automated Crypto API Pull\API.csv', mode = 'a', header = False)

#    df = pd.read_csv(r'C:\Users\Joven\Desktop\Portfolio\Automated Crypto API Pull\API.csv')

In [ ]:
# Automation

import os
from time import time
from time import sleep

# Creating the loop

for i in range(10):
    api_runner()
    print('API Completed Successfully')
    sleep(60)
exit()

In [ ]:
df

In [ ]:
# Simple number standardization

pd.set_option('display.float_format', lambda x: '%.5f' % x)

df

In [ ]:
# Displays the coin trends over time

df3 = df.groupby('name', sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()
df3

In [ ]:
# Need to chang the headers for better usability

df4 = df3.stack()
df4

In [ ]:
# Standardizing the format of the df

df5 = df4.to_frame(name = 'values')
df5

In [ ]:
df5.count()

In [ ]:
# Creating a new index

index = pd.Index(range(90))

df6 = df5.reset_index()
df6

In [ ]:
df7 = df6.rename(columns={'level_1': 'percent_change'})
df7

In [ ]:
# Standardizing percent_change column

df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d'],['1h','24h','7d','30d','60d','90d'])
df7

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Visualization for coin values over time

sns.catplot(x='percent_change', y='values', hue='name', data=df7, kind='point')

In [ ]:
# Creating visualization for just Bitcoin price changes

df10 = df[['name','quote.USD.price','timestamp']]
df10 = df10.query("name == 'Bitcoin'")
df10

In [ ]:
sns.set_theme(style="darkgrid")

sns.lineplot(x='timestamp', y='quote.USD.price', data = df10)